In [1]:
%pylab inline
import sep #http://sep.readthedocs.org/
import os 
from astropy.io import fits

Populating the interactive namespace from numpy and matplotlib


In [2]:
path_imagenes = '../imagenes_alineadas/deteccion_fuentes/'

In [3]:
visita = fits.open('../imagenes_alineadas/graficas/media.fits')

data = visita[0].data

shape_data = shape(data)
print shape_data

(1200, 1200)


In [5]:
#Background

try:
    bkg = sep.Background(data)
except ValueError:
    data = data.byteswap().newbyteorder()
    bkg = sep.Background(data)

In [12]:
#Source detection

thresh = 2.0 * bkg.globalrms
objects = sep.extract(data, thresh) # objects is a numpy structured array

num_objects = len(objects)
print num_objects # number of objects

182


In [7]:
objects_x = []
objects_y = []

for i in range(num_objects):
    x = objects['x'][i]
    y = objects['y'][i]
    
    objects_x.append(x)
    objects_y.append(y)
    
objects_x = array(objects_x)
objects_y = array(objects_y)

In [31]:
x_max = shape_data[0]
y_max = shape_data[1]

fig = figure(figsize=(14,14))
imshow(data, cmap='gray',vmin=0,vmax=7)
colorbar()

for i in range(len(objects)):
    x = objects_x[i]
    y = objects_y[i]
    
    porc_x = x/float(x_max)
    porc_y = 1-y/float(y_max)
    
    axvline(x,porc_y-0.02,porc_y+0.02,c='c')
    axhline(y,porc_x-0.02,porc_x+0.02,c='c') 
    text(x+1,y+1,str(i),color='c',fontsize=12)
    
savefig(path_imagenes+'fuentes.png', bbox_inches='tight')
close()

#Curvas de luz

In [16]:
path = '../../data_marcosmo/aligned/supernova_subtract/'
visitas = sort(os.listdir(path))

num_visitas = len(visitas)

In [20]:
print visitas

['+1.fits' '+3.fits' '-1,2.fits' '-2,2.fits' '-3.fits' '-4.fits' '0.fits']


In [17]:
objects_x_int = array([int(ox) for ox in objects_x])
objects_y_int = array([int(oy) for oy in objects_y])

In [18]:
curvas = zeros((num_objects,num_visitas))

In [23]:
for v in range(num_visitas):
            
    visita = fits.open(path+visitas[v])
    ccd14 = visita[1].data
    ccd14 = ccd14*65000/amax(ccd14) #Normalizando, no estoy segura si debería ir ahi
       
    for i in range(num_objects):
        
        columna = objects_x_int[i]
        fila = objects_y_int[i]
        
        pixel = ccd14[fila][columna]
            
        curvas[i][v] = pixel

    visita.close()

In [25]:
vis = arange(num_visitas)

for i in range(num_objects):
    intensidad = curvas[i][:]
    
    fig = figure(figsize=(20,10))
    scatter(vis,intensidad)
    plot(vis,intensidad,c='c')
    xlabel('Visita',fontsize=20)
    ylabel('Intensidad',fontsize=20)
    xlim(0,num_visitas-1)
    ylim(-amax(intensidad)*0.01,amax(intensidad)*1.01)
    savefig(path_imagenes+'curvas_de_luz/'+str(i)+'.png')
    close()

#Curva sobre fuentes

In [32]:
for i in range(num_objects):

    fila = objects_y_int[i]
    columna = objects_x_int[i]
    
    arreglo_pixel = zeros(23)
    arreglo_intensidad = zeros(23)
    indice_arreglos = 0
    
    if (fila-12 < 0):
        for lin in range(0,fila+11+1):
            pixel = data[lin][columna]
            arreglo_pixel[indice_arreglos] = lin
            arreglo_intensidad[indice_arreglos] = pixel
            indice_arreglos += 1
            
    elif (fila+12 > shape_data[1]):
        for lin in range(fila-11,shape_data[1]):
            pixel = data[lin][columna]
            arreglo_pixel[indice_arreglos] = lin
            arreglo_intensidad[indice_arreglos] = pixel
            indice_arreglos += 1

    else:
        for lin in range(fila-11,fila+11+1):
            pixel = data[lin][columna]
            arreglo_pixel[indice_arreglos] = lin
            arreglo_intensidad[indice_arreglos] = pixel
            indice_arreglos += 1
    
    fig = figure(figsize=(20,10))
    plot(arreglo_pixel,arreglo_intensidad,c='c')
    xlabel('Pixel',fontsize=20)
    ylabel('Intensidad',fontsize=20)
    savefig(path_imagenes+'curvas_sobre_fuentes/'+str(i)+'.png')
    close()